## Setup / Step 0

1. Load some basic libraries and set environment variables for AWS access
2. Initiate tileset to test

In [5]:
import eodc_hub_role
import morecantile
import numpy as np
import pandas as pd
from profiler.main import Timer
import sys
sys.path.append('..')
credentials = eodc_hub_role.fetch_and_set_credentials()

In [6]:
tms = morecantile.tms.get("WebMercatorQuad")
zooms = range(12) # Zoom 10 is the level at which you can see large roads, 15 is buildings
xyz_tiles = []
for z in zooms:
    tile = tms.tile(57, 28, z)
    xyz_tiles.append((tile.x, tile.y, tile.z))

xyz_tiles

[(0, 0, 0),
 (1, 0, 1),
 (2, 1, 2),
 (5, 3, 3),
 (10, 6, 4),
 (21, 13, 5),
 (42, 26, 6),
 (84, 53, 7),
 (168, 107, 8),
 (337, 214, 9),
 (674, 428, 10),
 (1348, 857, 11)]

# Profile titiler-xarray

In [11]:
%load_ext autoreload
%autoreload

import xarray_tile_reader
import zarr_reader
import zarr_helpers

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
shapes = ['180x360', '180x360', '569x1138', '1799x3599', '5689x11379', 'one_tenth_planet']

In [13]:
variable = 'data'
bucket = 'nasa-eodc-data-store'
datastores = {}
for shape in shapes:
    store_dir = f"s3://{bucket}/fake_data/store_{shape}.zarr"
    ds = zarr_helpers.open_dataset(store_dir)
    ds_specs = zarr_helpers.get_dataset_specs(shape, store_dir, variable, ds)
    datastores[shape] = ds_specs
    nochunk_store_dir = f"s3://{bucket}/fake_data/store_{shape}_nochunks.zarr"
    ds_nochunk = zarr_helpers.open_dataset(nochunk_store_dir)
    key = f"{shape}_nochunks"
    ds_nochunk_specs = zarr_helpers.get_dataset_specs(key, nochunk_store_dir, variable, ds_nochunk)
    datastores[key] = ds_nochunk_specs    

In [16]:
df = pd.DataFrame.from_dict(datastores, orient='index')
df.drop(columns=['collection_name', 'variable', 'source', 'dtype', 'number_coord_chunks', 'compression'])

,shape,lat_resolution,lon_resolution,chunk_size_mb,chunks
180x360,"{'time': 1, 'lat': 180, 'lon': 360}",1.005587,1.002786,0.247192,"{'time': 1, 'lat': 90, 'lon': 360}"
180x360_nochunks,"{'time': 1, 'lat': 180, 'lon': 360}",1.005587,1.002786,0.494385,"{'time': 1, 'lat': 180, 'lon': 360}"
569x1138,"{'time': 1, 'lat': 569, 'lon': 1138}",0.316901,0.316623,0.310936,"{'time': 1, 'lat': 143, 'lon': 285}"
569x1138_nochunks,"{'time': 1, 'lat': 569, 'lon': 1138}",0.316901,0.316623,4.940201,"{'time': 1, 'lat': 569, 'lon': 1138}"
1799x3599,"{'time': 1, 'lat': 1799, 'lon': 3599}",0.100111,0.100056,0.772476,"{'time': 1, 'lat': 225, 'lon': 450}"
1799x3599_nochunks,"{'time': 1, 'lat': 1799, 'lon': 3599}",0.100111,0.100056,49.397285,"{'time': 1, 'lat': 1799, 'lon': 3599}"
5689x11379,"{'time': 1, 'lat': 5689, 'lon': 11379}",0.031646,0.031640,1.933838,"{'time': 1, 'lat': 356, 'lon': 712}"
5689x11379_nochunks,"{'time': 1, 'lat': 5689, 'lon': 11379}",0.031646,0.031640,493.889854,"{'time': 1, 'lat': 5689, 'lon': 11379}"
one_tenth_planet,"{'time': 1, 'lat': 5689, 'lon': 11379}",0.010007,0.010005,1.933838,"{'time': 1, 'lat': 356, 'lon': 712}"
one_tenth_planet_nochunks,"{'time': 1, 'lat': 5689, 'lon': 11379}",0.010007,0.010005,493.889854,"{'time': 1, 'lat': 5689, 'lon': 11379}"


## Test different tiles

Below we test the time to generate tiles from each datastore 10 times.

In [ ]:
niters = 2
tile_results_df = {}
dataset_chunk_sizes = {}
results_df = datastores.copy()

for xyz_tile in xyz_tiles:
    results_df[xyz_tile] = {}
    for shape, data_specs in datastores.items():
        reference, multiscale = False, False

        results_df[xyz_tile][shape] = {
            'time to open (ms)': [],
            'rio reproject (ms)': [],
            'total time (ms)': []
        }
        timings_results = results_df[xyz_tile][shape]
        for iter in range(niters):
            with Timer() as t:
                image_and_timings, cprofile = xarray_tile_reader.tile(
                    data_specs['source'],
                    *xyz_tile,
                    variable=variable,
                )
            total_time = round(t.elapsed * 1000, 2)

            timings = image_and_timings[1]
            timings_results['time to open (ms)'].append(timings['time_to_open']),
            timings_results['rio reproject (ms)'].append(timings['rio.reproject']),
            timings_results['total time (ms)'].append(total_time)
        timings_results['mean time to open (ms)'] = np.mean(timings_results['time to open (ms)'])
        timings_results['mean rio reproject (ms)'] = np.mean(timings_results['rio reproject (ms)']) 
        timings_results['mean total time (ms)'] = np.mean(timings_results['total time (ms)'])

In [14]:
for xyz_tile in xyz_tiles:
    print(f"Results for {xyz_tile}")
    df = pd.DataFrame.from_dict(results_df[xyz_tile], orient='index')
    display(df.sort_values('mean total time (ms)'))
    print()

Results for (0, 0, 0)


,time to open (ms),rio reproject (ms),total time (ms),mean time to open (ms),mean rio reproject (ms),mean total time (ms)
180x360_nochunks,"[8.21, 6.38]","[18.11, 15.82]","[86.95, 78.74]",7.295,16.965,82.845
569x1138,"[6.41, 6.46]","[37.65, 38.66]","[102.65, 102.49]",6.435,38.155,102.570
180x360,"[9.57, 8.25]","[21.34, 20.68]","[112.92, 94.32]",8.910,21.010,103.620
569x1138_nochunks,"[7.86, 6.84]","[25.65, 20.96]","[91.88, 164.82]",7.350,23.305,128.350
1799x3599_nochunks,"[9.44, 6.84]","[113.15, 88.52]","[183.59, 156.3]",8.140,100.835,169.945
1799x3599,"[7.2, 6.72]","[178.98, 185.08]","[251.08, 252.62]",6.960,182.030,251.850
5689x11379,"[7.4, 7.1]","[1631.9, 1641.16]","[1702.06, 1711.03]",7.250,1636.530,1706.545
one_tenth_planet,"[9.48, 7.86]","[1784.4, 1554.77]","[1859.18, 1624.98]",8.670,1669.585,1742.080
one_tenth_planet_nochunks,"[10.06, 7.01]","[1954.96, 1786.05]","[2027.78, 1854.4]",8.535,1870.505,1941.090
5689x11379_nochunks,"[9.08, 7.88]","[2626.47, 1998.66]","[2705.16, 2077.29]",8.480,2312.565,2391.225



Results for (1, 0, 1)


,time to open (ms),rio reproject (ms),total time (ms),mean time to open (ms),mean rio reproject (ms),mean total time (ms)
180x360_nochunks,"[7.37, 6.41]","[16.77, 15.37]","[82.88, 77.73]",6.890,16.070,80.305
180x360,"[9.52, 6.49]","[17.32, 15.09]","[86.33, 78.68]",8.005,16.205,82.505
569x1138_nochunks,"[8.24, 6.24]","[21.88, 19.3]","[89.58, 84.23]",7.240,20.590,86.905
1799x3599,"[8.93, 6.58]","[60.09, 59.39]","[128.28, 127.31]",7.755,59.740,127.795
569x1138,"[8.9, 6.42]","[34.07, 23.04]","[100.25, 167.03]",7.660,28.555,133.640
1799x3599_nochunks,"[7.99, 7.15]","[93.98, 53.01]","[165.68, 121.05]",7.570,73.495,143.365
5689x11379,"[9.03, 8.03]","[405.0, 484.38]","[477.83, 561.57]",8.530,444.690,519.700
5689x11379_nochunks,"[8.69, 7.59]","[842.38, 835.53]","[921.0, 913.15]",8.140,838.955,917.075
one_tenth_planet,"[8.51, 8.36]","[1516.75, 1453.98]","[1685.62, 1531.92]",8.435,1485.365,1608.770
one_tenth_planet_nochunks,"[7.3, 7.84]","[1774.57, 1652.04]","[1850.61, 1729.04]",7.570,1713.305,1789.825



Results for (2, 1, 2)


...

,time to open (ms),rio reproject (ms),total time (ms),mean time to open (ms),mean rio reproject (ms),mean total time (ms)
180x360_nochunks,"[6.47, 6.69]","[15.75, 15.55]","[82.29, 81.47]",6.580,15.650,81.880
569x1138,"[6.46, 6.54]","[16.38, 15.92]","[83.19, 82.11]",6.500,16.150,82.650
5689x11379,"[7.34, 6.71]","[17.98, 16.59]","[85.53, 82.53]",7.025,17.285,84.030
180x360,"[6.88, 6.71]","[16.27, 15.64]","[86.62, 82.95]",6.795,15.955,84.785
569x1138_nochunks,"[6.34, 6.55]","[20.47, 17.49]","[86.12, 85.1]",6.445,18.980,85.610
one_tenth_planet,"[8.14, 7.51]","[19.51, 18.75]","[91.58, 90.07]",7.825,19.130,90.825
1799x3599,"[6.71, 7.4]","[16.63, 16.78]","[163.63, 88.78]",7.055,16.705,126.205
1799x3599_nochunks,"[6.6, 6.81]","[78.85, 44.4]","[147.23, 112.03]",6.705,61.625,129.630
5689x11379_nochunks,"[6.73, 7.2]","[671.92, 672.27]","[739.49, 743.88]",6.965,672.095,741.685
one_tenth_planet_nochunks,"[7.47, 7.43]","[663.43, 666.16]","[734.34, 817.37]",7.450,664.795,775.855



Results for (674, 428, 10)


,time to open (ms),rio reproject (ms),total time (ms),mean time to open (ms),mean rio reproject (ms),mean total time (ms)
1799x3599,"[6.46, 6.23]","[16.75, 14.07]","[81.24, 77.48]",6.345,15.410,79.360
180x360_nochunks,"[6.49, 6.39]","[15.17, 15.42]","[80.84, 80.42]",6.440,15.295,80.630
569x1138,"[6.35, 6.44]","[16.25, 15.93]","[81.54, 82.96]",6.395,16.090,82.250
569x1138_nochunks,"[6.53, 6.35]","[20.29, 17.09]","[86.63, 82.41]",6.440,18.690,84.520
5689x11379,"[7.16, 6.47]","[17.65, 17.59]","[86.44, 84.27]",6.815,17.620,85.355
one_tenth_planet,"[7.64, 6.68]","[20.22, 18.4]","[89.34, 84.81]",7.160,19.310,87.075
180x360,"[7.55, 7.45]","[17.72, 15.2]","[93.31, 82.13]",7.500,16.460,87.720
1799x3599_nochunks,"[6.95, 7.3]","[76.48, 45.62]","[220.78, 117.61]",7.125,61.050,169.195
one_tenth_planet_nochunks,"[6.46, 7.66]","[667.52, 670.66]","[735.8, 743.51]",7.060,669.090,739.655
5689x11379_nochunks,"[6.44, 7.26]","[678.19, 674.14]","[748.72, 747.11]",6.850,676.165,747.915



Results for (1348, 857, 11)


,time to open (ms),rio reproject (ms),total time (ms),mean time to open (ms),mean rio reproject (ms),mean total time (ms)
569x1138,"[7.11, 7.51]","[16.28, 16.73]","[87.82, 88.08]",7.310,16.505,87.950
1799x3599,"[7.25, 7.09]","[17.6, 18.07]","[88.79, 89.91]",7.170,17.835,89.350
569x1138_nochunks,"[7.75, 7.6]","[22.65, 17.32]","[95.68, 90.02]",7.675,19.985,92.850
one_tenth_planet,"[7.23, 8.58]","[20.21, 19.71]","[92.98, 96.98]",7.905,19.960,94.980
180x360_nochunks,"[7.78, 7.67]","[17.58, 15.28]","[100.53, 92.11]",7.725,16.430,96.320
5689x11379,"[7.66, 7.09]","[18.51, 17.03]","[88.38, 168.12]",7.375,17.770,128.250
1799x3599_nochunks,"[7.6, 7.37]","[78.74, 47.4]","[151.07, 117.5]",7.485,63.070,134.285
180x360,"[7.39, 7.11]","[19.8, 16.62]","[103.81, 181.32]",7.250,18.210,142.565
one_tenth_planet_nochunks,"[7.6, 7.31]","[670.97, 665.31]","[743.34, 741.03]",7.455,668.140,742.185
5689x11379_nochunks,"[7.53, 7.29]","[682.22, 682.54]","[757.53, 754.08]",7.410,682.380,755.805


In [32]:
# Dimensions of the grid
grid_width = 5689
grid_height = 11379

# Earth's circumference at the equator in kilometers
earth_circumference_km = 40075

# Calculate the extent of coverage area in kilometers
extent_width_km = earth_circumference_km / grid_width
extent_height_km = earth_circumference_km / grid_height

print(f"Resolution of the global dataset: {extent_width_km} km x {extent_height_km} km")

Resolution of the global dataset: 7.044296009843558 km x 3.5218384743826348 km
